# Translation

## 학습 목표

1. 다양한 RNN의 구성을 알아보기
2. 인코더와 디코더 구조의 필요성 이해하기
3. 교사 강요(teacher forcing)의 원리 알기
4. 훈련 단계와 추론 단계(inference)의 차이 알기

## 기계 번역 이야기

<img src = 'data/MT.png' height = '50%' width = '50%'>

* 규칙 기반 -> 통계적 기계 번역 -> 신경망 기계 번역
* seq2seq : 초창기 신경망 기계번역

## 시퀀스 처리하는 RNN

<img src = 'data/RNN.jpg' height = '50%' width = '50%'>

번역기는 첫번재 many to many 형식으로 만들어야 한다.

## seq2seq

### Encoder Decoder

<img src = 'data/seq2seq_1.jpg' height = '50%' width = '50%'>

* 2개의 RNN 아키텍쳐를 연결한 구조.
* 입력 문장을 받는 RNN을 인코더
* 두번째 RNN을 디코터

<img src = 'data/EncoderDecoder_1.png' height = '70%' width = '70%'>

* Encoder : Feature extractor 역할 - 어떤 데이터를 해석하기 위해 feature vector z를 만들어냅니다.
* Decoder : Feature z로부터 정보를 복원하여 다른 데이터를 재생성하는 역할.

* seq2seq 모델은 이 Encoder와 Decoder 둘 다 RNN 인 경우.
* feature vector : RNN이 문장을 해석해 만든 hidden state vector
* 문장 X를 z 라는 hidden state로 해석 후 z를 다시 언어 문장 Y롤 재생성
* 인코더에서 필요한 것은 인코더의 마지막 time step의 hidden state, 이를 두번째 RNN인 Decoder에게 전달

* 디코더는 인코더의 마지막 time step의 hidden state를 받아서 자신의 초기 hidden state로 하고 출력 문장을 생성.
* 여기서 특수 문자를 사용해서 출력 문장의 시작과 끝을 알려주기.
* 위 위 그림에서 GO, EOS 가 시작과 끝, 경우에 따라 SOS, EOS 라고도 함.

### Conditional Language Model

* 문장 생성기(Text Generator) : 언어 모델을 구현한 것
* 언어 모델 : n-1개의 단어 시퀀스 $w_1, ..., w_{n-1}$ 가 주어질때 n 번째 단어로 무엇이 올지 예측하는 확률 모델
* 파라미터 $\theta$를 모델링하는 언어 모델을 다음과 같이 표현
$$ P(w_n|w_1, ..., w_{n-1};\theta)$$
* 문장 생성기는 어떤 말을 만들고 싶은지 제어할 수 없다.


* 상황에 맞게 제어할 수 있게 언어모델을 확장하여 조건적 언어모델을 만듬.
$$ P(w_n|w_1, ..., w_{n-1},c;\theta)$$
* c : 아무 문장이나 만들지 말고 c에 맞는 문장을 만들어라.
* 인코더가 문장 X를 해석해서 *c* 를 만들고 *c* 를 문장생성기인 디코더에 입력으로 넣어주는 모델이 seq2seq

## 교사 강요(teacher forcing)

<img src = 'data/Teacher_forcing.png' height = '10%' width = '30%'>

* 훈련 과정에서 이전 time step이 잘못 예측하면 이번 예측도 잘못될 수 있고 이러면 훈련 시간이 엄청 길어진다.
* 이를 막기위해 이전 time step의 실제 값을 입력 값으로 사용하게 함.
* 이를 Teacher Forcing이라 한다.

## 단어 수준 VS 문자 수준

* Word level VS Character level
* time step의 입출력 단위
* 단어 집합은 엄청 많아지는 반면 문자는 알파벳 전부기 때문에 실제 구현 자체는 문자 수준이 더 편하다.
* 띄어쓰기 또는 많은 변종이 있는 언어는 문자 수준이 좋다.
* 하지만 그만큼 단어를 문자로 쪼개면서 데이터를 손실시키는것이기 때문에 충분한 데이터가 없다면 단어모델도 더 품질이 떨어진다.
* 최신 자연어처리는 subword를 기반으로 이루어진다.
* 우리는 문자 수준을 해보자.

# 번역기를 만들어보자

## 전처리

### Import Modules(library)

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras import layers
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np


ModuleNotFoundError: No module named 'tensorflow'

#### GPU

In [ ]:
from tensorflow.python.client import device_lib

print(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

### 파일 불러오기

In [ ]:
file_path = 'data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

### CC 제거 및 5만개만 사용

In [ ]:
lines = lines[['eng', 'fra']][:50000]
lines.sample(5)

### 시작과 종료 토큰 추가

In [ ]:
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

### 단어장 및 정수 인코딩 과정

* char_level = True : 문자단위로 Tokenizer 생성
* eng, fra의 각행에 토큰화를 수행
* 단어를 숫자값 인덱스로 변환하여 저장

In [ ]:
eng_tokenizer = Tokenizer(char_level=True)
fra_tokenizer = Tokenizer(char_level=True)

eng_tokenizer.fit_on_texts(lines.eng)
fra_tokenizer.fit_on_texts(lines.fra)

input_text = eng_tokenizer.texts_to_sequences(lines.eng)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)

print(input_text[:3])
print(target_text[:3])

* 단어장 크기를 변수로 저장. 0번 토큰을 생각해서 +1

In [ ]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

### 최대 길이 구해보기

In [ ]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

### 통계 정보

In [ ]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

### 인코더와 디코더의 데이터

* 인코더 입력인 영어와 달리 프랑스 시퀀스는 2가지 버전으로 나누어 준비
* 하나의 디코더 출력과 비교해야 할 정답 데이터로 사용할 원래 목적
* 나머지 하나는 teacher forcing 용(디코더 입력용)

* 디코더 입력은 <'eos'>가 필요없고 출력과 비교할 시퀀스는 <'sos'>가 필요없다.
* A를 B로 번역할때 훈련과정에서 디코더가 <'sos'>A 를 받아서 B<'eos'> 를 예측하도록 훈련

In [ ]:
encoder_input = input_text
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [ ]:
print(decoder_input[:3])
print(decoder_target[:3])

### Padding 진행

In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

In [ ]:
print(encoder_input[0])

역시 0으로 패딩이 되었습니다.

### 이제 정수에 따른 벡터화 방법으로 one-hot encoding 사용

* 샘플의 크기 = 샘플의 수 * 샘플의 크기 * 단어장의 크기

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

### Validation 데이터 분리

* 검증을 위해 5만 건 중에 3천건만 검증 데이터로 삼고, 나머지를 학습

In [ ]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

## 모델 훈련

[참고](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

### 추가 라이브러리 불러오기

In [ ]:
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.python.keras.models import Model

### 인코더 설계 

* LSTM 설계
* LSTM 은 hidden state, cell state 2개 존재
* Encoder LSTM 이 마지막 time step의 hidden state, cell state 를 Decoder LSTM 의 첫 번째 hidden state, cell state 로 전달

* 입력 텐서 생성.
* hidden size가 256인 인코더의 LSTM 셀 생성, return_state = True 를 통해 hidden state, cell state 를 return
* 디코더로 전달할 마지막 time step의 hidden state, cell state를 리턴. 
* hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.

In [ ]:
encoder_inputs = Input(shape=(None, eng_vocab_size))
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

### 디코더 설계 

* 입력 텐서 생성.
* hidden size가 256인 인코더의 LSTM 셀 생성
* decoder_outputs는 모든 time step의 hidden state
* initial_state 를 통해 초기 상태를 지정가능(이전의 encoder_states 사용)

In [ ]:
decoder_inputs = Input(shape=(None, fra_vocab_size))
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

### 디코더 출력층

* dense layer 에 softmax로 단어장에서 1개만 선택(문자 기준이므로)

In [ ]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

### 하나의 모델 만들기

* 인코더와 디코더를 연결

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

### 모델 훈련

In [ ]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
tf.test.gpu_device_name()

* batch_size = 128 인데 메모리가 부족해서 에러가 뜨므로 batch_size를 줄여야 하는데 cpu로 뜨네요 GPU를 한번 더 점검 해봅시다.

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)